In [2]:
import cython

time: 14.4 ms


In [4]:
%load_ext Cython
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 532 ms


In [5]:
%%cython
def sum(int a , int b):
    cdef int s = a + b
    return s

time: 2.21 s


In [6]:
def simple_sum(a, b):
    return a + b

time: 1.31 ms


In [8]:
%timeit sum(10, 20)

10000000 loops, best of 3: 90.9 ns per loop
time: 3.75 s
